# Econ 8210 Homework 1, Fall 2024
### Numerical Methods and Dynamic Programming
### Haosi Shen

## 1. Integration

Compute
\begin{equation*}
\int_{0}^{T} e^{-\rho t} u(1-e^{-\lambda t})\,dt
\end{equation*}
for $T=100$, $\rho = 0.04$, $\lambda = 0.02$, and $u(c)=-e^{-c}$ using **quadrature** (midpoint, Trapezoid, and Simpson rule) and Monte Carlo integration.

In [8]:
x = 1 + 1
x = x + 1
print(x)

3
